In [ ]:
'''
The content in notebook based on Ref but I try to convert them to Pyspark 
Ref: 
[1] Advanced Analytics with Spark, 2e
[2] https://github.com/novelari/advanced-analytics-spark
'''

In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as t
from pyspark.sql.types import StringType
from pyspark.sql.functions import split

In [2]:
config = SparkConf().setAll([('spark.driver.memory','10g'),('spark.executor.memory', '20g'), ('spark.executor.cores', '3'), ('spark.cores.max', '3')])
# config.set("spark.driver.memory", "10g")
# config.set("spark.executer.memory", "20g")

In [3]:
sc = SparkContext(appName= "Recommendation_sys", conf = config)
# sc.stop()

In [4]:
spark = SparkSession.builder\
                    .master('local')\
                    .appName("SparkSession_appName")\
                    .getOrCreate()

In [5]:
main = "C:/Users/New/Downloads/Data science in Action/Data set for spark/advance analytic with spark/data/chapter 3/"
# Spark.SQL_DF
# rawUserArtistData_ss = spark.read.option("delimiter"," ")\
#                                  .option("inferSchema","true")\
#                                  .option("charset", "utf-8")\
#                                  .csv(main + "user_artist_data.txt").cache()

# rawArtistData_ss = spark.read.option("delimiter",'\t')\
#                           .option("inferSchema","true")\
#                           .option("charset", "utf-8")\
#                           .csv(main + "artist_data.txt").cache()

# rawArtistAlias_ss = spark.read.option("delimiter",'\t')\
#                           .option("inferSchema","true")\
#                           .option("charset", "utf-8")\
#                           .csv(main + "artist_alias.txt").cache()

# print(type(rawUserArtistData_ss)) # SPARK_SQL_DF

In [6]:
# call data with SparkContext
rawUserArtistData = sc.textFile(main + "user_artist_data.txt").cache()
rawArtistData = sc.textFile(main + "artist_data.txt").cache()
rawArtistAlias = sc.textFile(main + "artist_alias.txt").cache()

print(type(rawUserArtistData)) # RDD

<class 'pyspark.rdd.RDD'>


In [7]:
UAD = rawUserArtistData 
AD = rawArtistData
AA = rawArtistAlias

In [8]:
# prevent effect of Null value 
def Int_checker(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

In [9]:
Int_checker('a')
# => when we use with filter the non-INT data will be remove Automatically

False

In [15]:
print('AD')
AD.take(5)

AD


['1134999\t06Crazy Life',
 '6821360\tPang Nakarin',
 '10113088\tTerfel, Bartoli- Mozart: Don',
 '10151459\tThe Flaming Sidebur',
 '6826647\tBodenstandig 3000']

In [16]:
new_AD = AD.map(lambda each_row: each_row.split('\t'))\
            .filter(lambda x: x[0] and Int_checker(x[0]))\
            .map(lambda x: (int(x[0]), x[1].strip()))

# non-INT data will be remove Automatically
# remove all the leading and trailing spaces from a string
new_AD.take(5)

[(1134999, '06Crazy Life'),
 (6821360, 'Pang Nakarin'),
 (10113088, 'Terfel, Bartoli- Mozart: Don'),
 (10151459, 'The Flaming Sidebur'),
 (6826647, 'Bodenstandig 3000')]

In [17]:
print('AA')
AA.take(5)

AA


['1092764\t1000311',
 '1095122\t1000557',
 '6708070\t1007267',
 '10088054\t1042317',
 '1195917\t1042317']

In [18]:
'''Mapping badID to goodID'''
new_AA = AA.map(lambda a: a.split('\t'))\
           .filter(lambda a: a[0] and Int_checker(a[0]))\
           .filter(lambda a: a[1] and Int_checker(a[1]))\
           .map(lambda a: (int(a[0]), int(a[1])))\
           .collectAsMap()

new_AA

{1092764: 1000311,
 1095122: 1000557,
 6708070: 1007267,
 10088054: 1042317,
 1195917: 1042317,
 1112006: 1000557,
 1187350: 1294511,
 1116694: 1327092,
 6793225: 1042317,
 1079959: 1000557,
 6789612: 1000591,
 1262241: 1000591,
 6791455: 1000591,
 6694867: 1000591,
 10141141: 1113738,
 1295140: 1000591,
 1027859: 1252408,
 2127019: 1000591,
 2153974: 1000591,
 1232342: 1000591,
 6968887: 1252408,
 1159841: 6826174,
 1195492: 1327092,
 7026209: 1246742,
 2012022: 1001590,
 1274042: 1327092,
 1073626: 668,
 2038625: 1001646,
 1026463: 1327652,
 2039670: 1001646,
 6798013: 1002414,
 1069294: 1009335,
 2056950: 1010728,
 1003908: 1011075,
 6607001: 1013391,
 2051033: 1014131,
 2136930: 1018491,
 2040025: 1029792,
 2123236: 1068687,
 1017615: 668,
 1037679: 1003105,
 2056553: 1103336,
 1182866: 1157715,
 2080828: 1334990,
 1235280: 1003468,
 1182267: 1003468,
 1029279: 1327810,
 2177612: 1338247,
 2023293: 1003468,
 1036318: 1003468,
 2116355: 1003468,
 1080440: 1003468,
 1202545: 1003681,

In [19]:
print('UAD')
UAD.take(5)

UAD


['1000002 1 55',
 '1000002 1000006 33',
 '1000002 1000007 8',
 '1000002 1000009 144',
 '1000002 1000010 314']

In [126]:
'''--------------------------------------------------RDD-Based API----------------------------------------------------------'''
from pyspark.mllib.recommendation import ALS, Rating

def buildRatings(rawUserArtistData, bArtistAlias):
    def getArtistRating(line):
        (userID, artistID, count) = map(lambda x: int(x), line.split(' '))
        try:
            finalArtistID = bArtistAlias.value[artistID]
        except KeyError:
            finalArtistID = artistID
        return Rating(userID, finalArtistID, count)

    return rawUserArtistData.map(lambda line: getArtistRating(line))

In [127]:
artistByID = new_AD
artistAlias = new_AA

In [128]:
trainData = buildRatings(UAD, bArtistAlias)
trainData.take(5)

[Rating(user=1000002, product=1, rating=55.0),
 Rating(user=1000002, product=1000006, rating=33.0),
 Rating(user=1000002, product=1000007, rating=8.0),
 Rating(user=1000002, product=1000009, rating=144.0),
 Rating(user=1000002, product=1000010, rating=314.0)]

In [163]:
model = ALS.trainImplicit(ratings=trainData, 
                          rank=50, 
                          iterations=10, 
                          lambda_=0.01, 
                          alpha=40.0)

In [130]:
type(model)

pyspark.mllib.recommendation.MatrixFactorizationModel

In [164]:
# trainData.unpersist()
# 50 User Features for 5 user
model.userFeatures().mapValues(lambda v: ", ".join( map(lambda x: str(x),v) )).take(5)

[(120,
  '0.01847374252974987, 0.1346728354692459, 0.21524611115455627, -0.031568821519613266, 0.05819370225071907, -0.0454770065844059, 0.21174047887325287, 0.06558344513177872, -0.12427325546741486, -0.21571150422096252, 0.14922748506069183, 0.1208685114979744, 0.04389527440071106, 0.013152672909200191, -0.381350576877594, -0.11419039964675903, 0.23068112134933472, 0.2718207538127899, -0.11577887088060379, 0.1131896823644638, -0.28084251284599304, -0.20395854115486145, 0.1309860497713089, 0.021665653213858604, 0.0406518317759037, 0.09064581990242004, 0.0280197374522686, -0.336208313703537, -0.07108686119318008, 0.11044036597013474, -0.21349027752876282, -0.11892475932836533, 0.3537682890892029, 0.07119128853082657, 0.06371480971574783, 0.15184816718101501, -0.06934674084186554, -0.11128142476081848, 0.14201277494430542, -0.03019457682967186, -0.078889861702919, 0.0522034727036953, -0.18735453486442566, 0.11577180027961731, 0.06402997672557831, -0.07317326217889786, 0.0434241369366645

In [165]:
userID = 2093760 # we named this user Sara
recommendations = model.recommendProducts(userID, 5)

recommendations

Rating(user=2093760, product=1001819, rating=0.32046294844618234)
Rating(user=2093760, product=1003249, rating=0.32014321506119925)
Rating(user=2093760, product=1300642, rating=0.31997051561678613)
Rating(user=2093760, product=1007614, rating=0.3199046526287555)
Rating(user=2093760, product=829, rating=0.31983416161366623)


In [194]:
recommended_list = [1001819, 1003249, 1300642, 1007614, 892]

In [195]:
recommendedArtists = artistByID.filter(lambda artist: artist[0] in recommended_list).collect()

recommendedArtists

[(1007614, 'Jay-Z'),
 (1003249, 'Ludacris'),
 (1001819, '2Pac'),
 (892, 'Williams Traffic'),
 (1300642, 'The Game')]

#### Let's see what artist are Sara prefer

In [196]:
rawArtistsForUser = rawUserArtistData.map(lambda x: x.split(' '))\
                                     .filter(lambda x: int(x[0]) == userID)

In [197]:
existingProducts = rawArtistsForUser.map(lambda x: int(x[1])).collect()
existingProducts

[1180, 1255340, 378, 813, 942]

In [198]:
existingArtists = artistByID.filter(lambda artist: artist[0] in existingProducts).collect()
print('Sara prefers:')
for val in existingArtists:
    print(val)
    

Sara prefers:
(1180, 'David Gray')
(378, 'Blackalicious')
(813, 'Jurassic 5')
(1255340, 'The Saw Doctors')
(942, 'Xzibit')


#### Next step we need to analyze what relationship between existingArtists and recommened Artist.

In [20]:
'''----------------------------------------High level API---------------------------------------'''
'''Convert RDD(pipeline) to Spark-DF'''
from  pyspark.sql.types import Row

def f(x):
    d = {}
    for i in range(len(x)):
        d[str(i)] = x[i]
    return d

# EX
# seq = [['a','b','c'],
#        ['d','e','f'] ]
# print( f(seq) )
# print( Row(**f(seq)) )

In [21]:
def buildCount(rawUserArtistData, bArtistAlias):
    
    def getArtistCount(line):
        (userID, artistID, count) = map(lambda x: int(x), line.split(' '))    
        try:
            CorrectArtistID = bArtistAlias.value[artistID] # to map bad artist id to good artist id
        except KeyError:
            CorrectArtistID = artistID
        return (userID, CorrectArtistID, count)

    return rawUserArtistData.map(lambda line: getArtistCount(line))

In [22]:
# sc.broadcast use to spread parameter to all excecutor
# After we apply sc.broadcast, we can use .value method to see the dict value.
bArtistAlias = sc.broadcast(new_AA)
# bArtistAlias.value[1092764]
trainData_HLV = buildCount(UAD, bArtistAlias)

In [23]:
new_AD_df = new_AD.map(lambda x: Row(**f(x))).toDF()
mapping = dict(zip(['0', '1'], ['artist_id', 'artist_name']))
new_AD_df = new_AD_df.select([F.col(c).alias(mapping.get(c)) for c in new_AD_df.columns])

new_AD_df.show(5)

+---------+--------------------+
|artist_id|         artist_name|
+---------+--------------------+
|  1134999|        06Crazy Life|
|  6821360|        Pang Nakarin|
| 10113088|Terfel, Bartoli- ...|
| 10151459| The Flaming Sidebur|
|  6826647|   Bodenstandig 3000|
+---------+--------------------+
only showing top 5 rows



In [24]:
trainData_df = trainData_HLV.map(lambda x: Row(**f(x))).toDF()
mapping = dict(zip(['0', '1', '2'], ['user_id', 'artist_id', 'playcount']))
trainData_df = trainData_df.select([F.col(c).alias(mapping.get(c)) for c in trainData_df.columns]) # get 0,1,2 as Keys

trainData_df.show(5)

+-------+---------+---------+
|user_id|artist_id|playcount|
+-------+---------+---------+
|1000002|        1|       55|
|1000002|  1000006|       33|
|1000002|  1000007|        8|
|1000002|  1000009|      144|
|1000002|  1000010|      314|
+-------+---------+---------+
only showing top 5 rows



## The ALS Model in Spark contains the following helpful methods:

#### recommendForAllItems(int numUsers)
Returns top numUsers users recommended for each item, for all items.

#### recommendForAllUsers(int numItems)
Returns top numItems items recommended for each user, for all users.

#### recommendForItemSubset(Dataset<?> dataset, int numUsers)
Returns top numUsers users recommended for each item id in the input data set.

#### recommendForUserSubset(Dataset<?> dataset, int numItems)
Returns top numItems items recommended for each user id in the input data set.

In [64]:
userID = 2093760
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

model = ALS(implicitPrefs=True,
            rank=50,
            regParam=0.01, 
            alpha=40.0,
            maxIter=10,
            userCol="user_id", 
            itemCol="artist_id", 
            ratingCol="playcount")

# rank = number of features.

# # OR
# model = ALS()
# (alsEstimator.setRank(10)
#   .setUserCol("userid")
#   .setItemCol("artistid")
#   .setRatingCol("playcount")
#   .setMaxIter(5))

In [65]:
model_fit = model.fit(trainData_df)

In [29]:
# What Sara have been listened
trainData_df.filter(trainData_df['user_id'] == userID).show()

+-------+---------+---------+
|user_id|artist_id|playcount|
+-------+---------+---------+
|2093760|     1180|        1|
|2093760|  1255340|        3|
|2093760|      378|        1|
|2093760|      813|        2|
|2093760|      942|        7|
+-------+---------+---------+



In [66]:
model_fit.userFactors.show(1, truncate=False)

+---+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |features                                                                                                                                                                                                                                                                                                                                                             

In [62]:
# test
trainData_df.select(model.getUserCol()).distinct().limit(2).show()
# model.getUserCol() equivalent to trainData_df['user_id']

+-------+
|user_id|
+-------+
|1000061|
|1000070|
+-------+



In [55]:
trainData_df.createOrReplaceTempView('trainData_df_SQL')

In [59]:
# I prefer this one. It is far better T__T
sara = spark.sql('SELECT DISTINCT user_id \
           FROM trainData_df_SQL \
           WHERE user_id = 2093760'
         )

In [67]:
recom_user = model_fit.recommendForUserSubset(sara, 5)

In [63]:
recom_user.show(truncate=False)

+-------+----------------------------------------------------------------------------------------------------------------+
|user_id|recommendations                                                                                                 |
+-------+----------------------------------------------------------------------------------------------------------------+
|2093760|[[1300642, 0.028670812], [4605, 0.028028714], [829, 0.027974127], [1037970, 0.02780246], [1001819, 0.027772292]]|
+-------+----------------------------------------------------------------------------------------------------------------+

